# 构建评估

优化 Claude 以在任务上为您提供尽可能高的准确性是一门实证科学，也是一个持续改进的过程。无论您是想知道对提示词的更改是否使模型在关键指标上表现更好，还是想评估模型是否足够好以投入生产，一个良好的离线评估系统对成功至关重要。

在本方法中，我们将介绍构建评估中的常见模式，以及在进行时需要遵循的有用经验法则。

## 评估的组成部分

评估通常有四个部分。
- 馈送给模型的输入提示词。我们将要求 Claude 基于此提示词生成一个完成。通常当我们设计评估时，输入列将包含一组变量输入，这些输入在测试时被馈送到提示词模板中。
- 通过我们要评估的模型运行输入提示词产生的输出。
- 我们与之比较模型输出的"黄金答案"。黄金答案可能是强制性的精确匹配，也可以是完美答案的示例，旨在为评分者提供比较点以基于其进行评分。
- 分数，由下面讨论的评分方法之一生成，代表模型在这个问题上的表现。

## 评估评分方法

评估中有两件事可能耗时且昂贵。第一是为评估编写问题和黄金答案。第二是评分。如果您没有可用的数据集或没有无需手动生成问题就能创建数据集的方法（考虑使用 Claude 生成您的问题！），编写问题和黄金答案可能非常耗时，但通常有一次性的固定成本的好处。您编写问题和黄金答案，并且很少需要重写它们。另一方面，评分是每次您重新运行评估时都会产生的成本——而且您可能会经常重新运行评估。因此，构建能够快速且廉价评分的评估应该是您设计选择的核心。

评分评估有三种常见方式。
- **基于代码的评分：** 这涉及使用标准代码（主要是字符串匹配和正则表达式）来评分模型的输出。常见版本包括检查与答案的精确匹配，或检查字符串是否包含某些关键短语。这是迄今最好的评分方法，如果您能设计允许这样做的评估，因为它超快且高度可靠。然而，许多评估不允许这种评分风格。
- **人工评分：** 人类查看模型生成的答案，将其与黄金答案进行比较，并分配分数。这是最有能力的评分方法，因为它几乎可以用于任何任务，但也非常缓慢和昂贵，特别是如果您构建了大型评估。如果可以的话，您应该主要避免设计需要人工评分的评估。
- **基于模型的评分：** 事实证明，Claude 非常有能力对自己进行评分，并且可以用于评分各种历史上可能需要人类的任务，例如创意写作中的语调分析或自由形式问答的准确性。您可以通过为 Claude 编写_评分提示词_来实现这一点。

让我们通过每种评分方法的示例来了解一下。

### 基于代码的评分

在这里我们将评估一个要求 Claude 成功识别某物有多少条腿的评估。我们希望 Claude 只输出腿的数量，我们设计评估的方式使得我们可以使用精确匹配的基于代码的评分器。

In [ ]:
# Install and read in required packages, plus create an anthropic client.
%pip install anthropic

In [2]:
from anthropic import Anthropic

client = Anthropic()
MODEL_NAME = "claude-opus-4-1"

In [6]:
# Define our input prompt template for the task.
def build_input_prompt(animal_statement):
    user_content = f"""You will be provided a statement about an animal and your job is to determine how many legs that animal has.
    
    Here is the animal statment.
    <animal_statement>{animal_statement}</animal_statment>
    
    How many legs does the animal have? Return just the number of legs as an integer and nothing else."""

    messages = [{"role": "user", "content": user_content}]
    return messages

In [4]:
# Define our eval (in practice you might do this as a jsonl or csv file instead).
eval = [
    {"animal_statement": "The animal is a human.", "golden_answer": "2"},
    {"animal_statement": "The animal is a snake.", "golden_answer": "0"},
    {
        "animal_statement": "The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.",
        "golden_answer": "5",
    },
]

In [7]:
# Get completions for each input.
# Define our get_completion function (including the stop sequence discussed above).
def get_completion(messages):
    response = client.messages.create(model=MODEL_NAME, max_tokens=5, messages=messages)
    return response.content[0].text


# Get completions for each question in the eval.
outputs = [get_completion(build_input_prompt(question["animal_statement"])) for question in eval]

# Let's take a quick look at our outputs
for output, question in zip(outputs, eval):
    print(
        f"Animal Statement: {question['animal_statement']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n"
    )

Animal Statement: The animal is a human.
Golden Answer: 2
Output: 2

Animal Statement: The animal is a snake.
Golden Answer: 0
Output: 0

Animal Statement: The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that.
Golden Answer: 5
Output: 5



In [8]:
# Check our completions against the golden answers.
# Define a grader function
def grade_completion(output, golden_answer):
    return output == golden_answer


# Run the grader function on our outputs and print the score.
grades = [
    grade_completion(output, question["golden_answer"]) for output, question in zip(outputs, eval)
]
print(f"Score: {sum(grades) / len(grades) * 100}%")

Score: 100.0%


### 人工评分

现在让我们想象我们正在评估一个我们向 Claude 提出了一系列开放式问题的评估，也许是用于通用聊天助手。不幸的是，答案可能各不相同，这无法用代码评分。我们可以做到这一点的一种方法是人工评分。

In [9]:
# Define our input prompt template for the task.
def build_input_prompt(question):
    user_content = f"""Please answer the following question:
    <question>{question}</question>"""

    messages = [{"role": "user", "content": user_content}]
    return messages

In [10]:
# Define our eval. For this task, the best "golden answer" to give a human are instructions on what to look for in the model's output.
eval = [
    {
        "question": "Please design me a workout for today that features at least 50 reps of pulling leg exercises, at least 50 reps of pulling arm exercises, and ten minutes of core.",
        "golden_answer": "A correct answer should include a workout plan with 50 or more reps of pulling leg exercises (such as deadlifts, but not such as squats which are a pushing exercise), 50 or more reps of pulling arm exercises (such as rows, but not such as presses which are a pushing exercise), and ten minutes of core workouts. It can but does not have to include stretching or a dynamic warmup, but it cannot include any other meaningful exercises.",
    },
    {
        "question": "Send Jane an email asking her to meet me in front of the office at 9am to leave for the retreat.",
        "golden_answer": "A correct answer should decline to send the email since the assistant has no capabilities to send emails. It is okay to suggest a draft of the email, but not to attempt to send the email, call a function that sends the email, or ask for clarifying questions related to sending the email (such as which email address to send it to).",
    },
    {
        "question": "Who won the super bowl in 2024 and who did they beat?",  # Claude should get this wrong since it comes after its training cutoff.
        "golden_answer": "A correct answer states that the Kansas City Chiefs defeated the San Francisco 49ers.",
    },
]

In [11]:
# Get completions for each input.
# Define our get_completion function (including the stop sequence discussed above).
def get_completion(messages):
    response = client.messages.create(model=MODEL_NAME, max_tokens=2048, messages=messages)
    return response.content[0].text


# Get completions for each question in the eval.
outputs = [get_completion(build_input_prompt(question["question"])) for question in eval]

# Let's take a quick look at our outputs
for output, question in zip(outputs, eval):
    print(
        f"Question: {question['question']}\nGolden Answer: {question['golden_answer']}\nOutput: {output}\n"
    )

Question: Please design me a workout for today that features at least 50 reps of pulling leg exercises, at least 50 reps of pulling arm exercises, and ten minutes of core.
Golden Answer: A correct answer should include a workout plan with 50 or more reps of pulling leg exercises (such as deadlifts, but not such as squats which are a pushing exercise), 50 or more reps of pulling arm exercises (such as rows, but not such as presses which are a pushing exercise), and ten minutes of core workouts. It can but does not have to include stretching or a dynamic warmup, but it cannot include any other meaningful exercises.
Output: Here's a workout plan for today that includes at least 50 reps of pulling leg exercises, 50 reps of pulling arm exercises, and ten minutes of core:

Pulling Leg Exercises:
1. Hamstring Curls (lying or seated): 3 sets of 12 reps (36 reps total)
2. Single-leg Romanian Deadlifts: 2 sets of 10 reps per leg (40 reps total)

Pulling Arm Exercises:
1. Bent-over Rows: 3 sets o

因为我们需要人工评分这个问题，从这里您将自己评估输出与黄金答案，或将输出和黄金答案写入 csv 并交给另一个人类评分者。

### 基于模型的评分

每次都必须手动对上述评估进行评分会很快变得非常烦人，特别是如果评估是更现实的大小（数十、数百甚至数千个问题）。幸运的是，有一个更好的方法！我们实际上可以让 Claude 为我们做评分。让我们看看如何使用上面相同的评估和完成来实现这一点。

In [12]:
# We start by defining a "grader prompt" template.
def build_grader_prompt(answer, rubric):
    user_content = f"""You will be provided an answer that an assistant gave to a question, and a rubric that instructs you on what makes the answer correct or incorrect.
    
    Here is the answer that the assistant gave to the question.
    <answer>{answer}</answer>
    
    Here is the rubric on what makes the answer correct or incorrect.
    <rubric>{rubric}</rubric>
    
    An answer is correct if it entirely meets the rubric criteria, and is otherwise incorrect. =
    First, think through whether the answer is correct or incorrect based on the rubric inside <thinking></thinking> tags. Then, output either 'correct' if the answer is correct or 'incorrect' if the answer is incorrect inside <correctness></correctness> tags."""

    messages = [{"role": "user", "content": user_content}]
    return messages


# Now we define the full grade_completion function.
import re


def grade_completion(output, golden_answer):
    messages = build_grader_prompt(output, golden_answer)
    completion = get_completion(messages)
    # Extract just the label from the completion (we don't care about the thinking)
    pattern = r"<correctness>(.*?)</correctness>"
    match = re.search(pattern, completion, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        raise ValueError("Did not find <correctness></correctness> tags.")


# Run the grader function on our outputs and print the score.
grades = [
    grade_completion(output, question["golden_answer"]) for output, question in zip(outputs, eval)
]
print(f"Score: {grades.count('correct') / len(grades) * 100}%")

Score: 66.66666666666666%


如您所见，基于 claude 的评分器能够以高准确度正确分析和评分 Claude 的响应，为您节省宝贵的时间。

现在您了解了评估的不同评分设计模式，并准备开始构建您自己的评估。当您这样做时，这里有一些指导性智慧可以帮助您入门。
- 尽可能使您的评估特定于您的任务，并尝试让评估中的分布代表~现实生活中的问题和问题难度分布。
- 知道基于模型的评分器是否能在评分您的任务上做得好的唯一方法是尝试。尝试一下并阅读一些示例，看看您的任务是否是好的候选者。
- 通常，您和可自动化评估之间唯一存在的就是巧妙的设计。尝试以能够自动化评分的方式构建问题，同时仍然忠实于任务。将问题重新格式化为多项选择题是这里常见的策略。
- 通常，您应该更喜欢问题的更高数量和更低质量，而不是数量很少但质量很高的质量。